### **TRAINING MODEL**

**Hashing Trick**

In [ ]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
data=pd.read_csv("/content/drive/MyDrive/archive/avazu_4m.txt",sep=',')

In [ ]:
data=data.drop_duplicates()

In [ ]:
data.columns

Index(['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21'],
      dtype='object')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4042896 entries, 0 to 4042895
Data columns (total 22 columns):
 #   Column            Dtype  
---  ------            -----  
 0   id                float64
 1   hour              int64  
 2   C1                int64  
 3   banner_pos        int64  
 4   site_id           uint64 
 5   site_domain       uint64 
 6   site_category     uint64 
 7   app_id            uint64 
 8   app_domain        uint64 
 9   app_category      uint64 
 10  device_id         uint64 
 11  device_ip         uint64 
 12  device_type       int64  
 13  device_conn_type  int64  
 14  C14               int64  
 15  C15               int64  
 16  C16               int64  
 17  C17               int64  
 18  C18               int64  
 19  C19               int64  
 20  C20               int64  
 21  C21               int64  
dtypes: float64(1), int64(13), uint64(8)
memory usage: 678.6 MB


In [ ]:
import hashlib
from typing import Tuple

from sklearn.base import BaseEstimator, TransformerMixin


def hash_modulo(val, mod):
    md5 = hashlib.md5()  # can be other deterministic hash functions
    md5.update(str(val).encode())
    return int(md5.hexdigest(), 16) % mod


class FeatureHasher(BaseEstimator, TransformerMixin):
    def __init__(self, num_buckets: int):
        self.num_buckets = num_buckets

    def fit(self, X: pd.Series):
        return self

    def transform(self, X: pd.Series):
        return X.apply(lambda x: hash_modulo(x, self.num_buckets))


fh = FeatureHasher(num_buckets=2**20)
for i in data.columns:
  if i!='click':
    data[i]=fh.transform(data[i])

In [ ]:
data

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,574155,0,692291,664865,484570,678021,757257,849080,47018,824059,...,361627,296492,241542,506242,777226,989235,296492,314358,535688,638408
1,870699,0,662755,664865,484570,33225,350458,322905,108875,494310,...,361627,484570,588336,506242,777226,125992,506611,357765,1001792,9617
2,10509,0,382709,664865,484570,150680,524746,849080,47018,824059,...,361627,484570,291583,506242,777226,961580,506611,752408,535688,180363
3,71416,0,473497,664865,484570,33225,350458,322905,742748,614770,...,361627,484570,1027696,506242,777226,143666,484570,353213,70363,791671
4,226595,0,206975,664865,484570,968974,784352,782984,47018,824059,...,361627,484570,457834,506242,777226,358491,484570,752408,544314,52871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4042891,288016,1,341831,664865,484570,678021,757257,849080,47018,824059,...,361627,484570,409423,506242,777226,870668,484570,752408,535688,791671
4042892,719265,1,589657,664865,484570,678021,757257,849080,47018,824059,...,361627,484570,626443,506242,777226,255114,484570,752408,535688,952535
4042893,1021387,0,238380,664865,484570,284028,788651,782984,47018,824059,...,361627,484570,246609,506242,777226,999963,506611,291578,67997,633549
4042894,730592,0,245598,664865,484570,674057,741711,782984,47018,824059,...,361627,484570,689619,506242,777226,999963,506611,291578,832457,633549


In [ ]:
data.to_csv("/content/drive/MyDrive/archive/data_hash.csv")

**CTR Model**

In [ ]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
data=pd.read_csv("/content/drive/MyDrive/archive/data_hash.csv")
data=data.drop(['Unnamed: 0','id'],axis=1)

In [ ]:
data

,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
1166677,1,281309,664865,361627,922502,616254,782984,47018,824059,607188,...,361627,484570,455761,506242,777226,39606,506611,594748,535688,633549
606445,1,841084,664865,484570,678021,757257,849080,47018,824059,607188,...,361627,296492,417758,506242,777226,255114,484570,752408,236504,952535
168209,1,96413,664865,484570,977101,501284,1022116,47018,824059,607188,...,361627,484570,673499,301196,489048,1036838,296492,752408,535688,633549
2599142,1,925841,664865,484570,678021,757257,849080,47018,824059,607188,...,361627,484570,293587,506242,777226,255114,484570,752408,535688,952535
1164154,1,901247,664865,484570,678021,757257,849080,47018,824059,607188,...,361627,296492,241542,506242,777226,989235,296492,314358,535688,638408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1679379,0,274771,664865,361627,90585,144770,782984,47018,824059,607188,...,361627,484570,689619,506242,777226,999963,506611,291578,67997,633549
2620754,0,868450,664865,484570,33225,350458,322905,513178,270591,565080,...,361627,296492,1037589,506242,777226,609016,484570,640543,554069,791671
1964015,0,868633,664865,484570,1021468,186561,782984,47018,824059,607188,...,361627,484570,926768,506242,777226,59196,506611,999383,535688,633549
3226906,0,593612,664865,484570,33225,350458,322905,882771,270591,898039,...,361627,484570,359933,506242,777226,574558,506611,758142,535688,997583


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n - 1)

In [ ]:
X_train

,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
586045,0,656984,664865,484570,678021,757257,849080,47018,824059,607188,...,361627,484570,582600,506242,777226,998337,484570,891675,236504,791671
3863606,0,956543,664865,361627,90585,144770,782984,47018,824059,607188,...,361627,484570,966911,506242,777226,501720,484570,291578,535688,791671
457529,0,363543,664865,484570,678021,757257,849080,47018,824059,607188,...,361627,484570,241522,506242,777226,255114,484570,752408,535688,952535
1805172,0,278205,664865,484570,33225,350458,322905,513178,270591,565080,...,361627,296492,652555,506242,777226,609016,484570,640543,554069,791671
1308702,0,245598,664865,361627,144978,616254,782984,47018,824059,607188,...,361627,484570,569541,506242,777226,69890,506611,562613,1008392,9617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2356330,0,328532,664865,361627,956465,616254,782984,47018,824059,607188,...,361627,484570,249723,506242,777226,473298,506611,132968,1008392,809126
3511566,0,209378,664865,361627,922502,616254,782984,47018,824059,607188,...,361627,484570,435949,506242,777226,39606,506611,594748,1024634,633549
2229084,0,791129,664865,484570,33225,350458,322905,755368,942345,898039,...,361627,484570,866520,506242,777226,1032607,361627,53755,96183,464148
2768307,0,36021,135840,484570,465473,661096,322905,47018,824059,607188,...,484570,484570,115859,506242,777226,775197,484570,1046416,535688,245195


In [ ]:
import math
D= 2**20+100
a=0.1
w=np.zeros(D)
n=np.zeros(D)

for v in X_train.values:
  s=0
  i=0
  for f in v:
    if i ==0:
      i=i+1
      continue
    index = int(f)+1
    s=s+int(w[index])
    i=i+1
  try:
    p=1/(1+ math.exp(s))
  except OverflowError:
    p=1/(1+ float('inf'))
  i=0
  for f in v:
    if i ==0:
      i=i+1
      continue
    # print(s)
    # print(p)
    # print(str(w[f]-a*(p-v[1])/math.sqrt(n[int(f)]+1)))
    # print('/////')
    w[f]=w[f]-a*(p-v[0])/math.sqrt(n[int(f)]+1)
    n[int(f)]=n[int(f)]+1
    i=i+1

In [ ]:
w

array([-0.16158353, -0.05      , -0.05      , ...,  0.        ,
        0.        ,  0.        ])

In [ ]:
import math
y_pred=[]
y_predd=[]
for v in X_test.values:
  s=0
  i=0
  for f in v:
    if i ==1:
      i=i+1
      continue
    index = int(f)+1
    s=s+int(w[index])
    i=i+1
  try:
    p=1/(1+ math.exp(s))
  except OverflowError:
    p=1/(1+ float('inf'))
  y_pred.append(round(p))
  y_predd.append(p)


In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(X_test['click'], y_pred)

In [ ]:
accuracy

0.817842390363353

In [ ]:
from sklearn.metrics import log_loss
log_loss(X_test['click'], y_predd)

0.7287757483544941

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(X_test['click'], y_predd)

0.2567193211433944

In [ ]:
from sklearn.metrics import f1_score
f1_score(X_test['click'], y_pred)

0.03217137037158721

In [ ]:
with open('/content/drive/MyDrive/archive/w_parameter.npy', 'wb') as f:
  np.save(f, w)

In [ ]:
with open('/content/drive/MyDrive/archive/w_parameter.npy', 'rb') as f:
  w = np.load(f)

**DNN Model**

In [ ]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
data=pd.read_csv("/content/drive/MyDrive/archive/data_hash.csv")
data=data.drop(['Unnamed: 0','id'],axis=1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
y=data['click']
data=data.drop(['click'],axis=1)
scaler = MinMaxScaler()
datai = scaler.fit_transform(data)
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test = train_test_split(datai,y, test_size=0.2, random_state=42)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow import keras
from keras import optimizers
model = Sequential()
model.add(Dense(60, input_dim=X_train.shape[1], activation='relu'))
# model.add(Dense(100, activation='relu'))
# model.add(Dense(100, activation='relu'))
# model.add(Dense(80, activation='relu'))
# model.add(Dense(60, activation='relu'))
# model.add(Dense(20, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['acc'])
callback = keras.callbacks.EarlyStopping(monitor='loss',patience=3)
model.fit(X_train, y_train, epochs=30, batch_size=128,callbacks=[callback],verbose=1)

Epoch 1/30
25269/25269 [==============================] - 90s 3ms/step - loss: 0.4264 - acc: 0.8304
Epoch 2/30
25269/25269 [==============================] - 84s 3ms/step - loss: 0.4206 - acc: 0.8312
Epoch 3/30
25269/25269 [==============================] - 85s 3ms/step - loss: 0.4186 - acc: 0.8315
Epoch 4/30
25269/25269 [==============================] - 90s 4ms/step - loss: 0.4174 - acc: 0.8316
Epoch 5/30
25269/25269 [==============================] - 91s 4ms/step - loss: 0.4165 - acc: 0.8317
Epoch 6/30
25269/25269 [==============================] - 88s 3ms/step - loss: 0.4158 - acc: 0.8318
Epoch 7/30
25269/25269 [==============================] - 84s 3ms/step - loss: 0.4154 - acc: 0.8319
Epoch 8/30
25269/25269 [==============================] - 98s 4ms/step - loss: 0.4150 - acc: 0.8320
Epoch 9/30
25269/25269 [==============================] - 92s 4ms/step - loss: 0.4148 - acc: 0.8320
Epoch 10/30
25269/25269 [==============================] - 86s 3ms/step - loss: 0.4145 - acc: 0.8321

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
from sklearn.metrics import log_loss
print('logarithmic loss: '+str(log_loss(y_test, y_pred)))
from sklearn.metrics import mean_squared_error
print('root-mean squared error: '+str(mean_squared_error(y_test, y_pred)))
y_pred = [int(round(x[0])) for x in y_pred]
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))
from sklearn.metrics import f1_score
print('f1 score: '+str(f1_score(y_test,y_pred)))

25269/25269 [==============================] - 56s 2ms/step
logarithmic loss: 0.4109585387641887
root-mean squared error: 0.12814966472861924
Accuracy: 83.34%
f1 score: 0.08170582902172506


In [ ]:
from tensorflow.keras.models import load_model
# load model
model=load_model('/content/drive/MyDrive/archive/dnn2.h5')
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 60)                1380      
                                                                 
 dense_31 (Dense)            (None, 10)                610       
                                                                 
 dense_32 (Dense)            (None, 10)                110       
                                                                 
 dense_33 (Dense)            (None, 10)                110       
                                                                 
 dense_34 (Dense)            (None, 1)                 11        
                                                                 
Total params: 2221 (8.68 KB)
Trainable params: 2221 (8.68 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.save('/content/drive/MyDrive/archive/dnn2.h5')
print('Model Saved!')

Model Saved!
